# Music Generation with LSTM model

## Imports

In [1]:

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.utils import to_categorical
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint
from music21 import converter, note, chord
import os
import pickle
import numpy as np

2024-01-05 00:06:32.917473: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-05 00:06:32.944259: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 00:06:32.944291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 00:06:32.944969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 00:06:32.949497: I tensorflow/core/platform/cpu_feature_guar

## Load Chopin MIDI files

In [2]:
data_folder = 'data'
files = [os.path.join(data_folder, file) for file 
         in os.listdir(data_folder) if file.endswith('.mid')]

In [3]:
files

['data/chpn-p4.mid',
 'data/chpn-p14.mid',
 'data/chpn-p3.mid',
 'data/chpn-p7.mid',
 'data/chpn_op66.mid',
 'data/chpn_op25_e1.mid',
 'data/chpn_op27_2.mid',
 'data/chpn_op25_e4.mid',
 'data/chpn_op7_2.mid',
 'data/chpn_op7_1.mid',
 'data/chpn-p10.mid',
 'data/chpn_op25_e3.mid',
 'data/chpn_op25_e2.mid',
 'data/chpn-p5.mid',
 'data/chpn_op35_4.mid',
 'data/chpn_op10_e05.mid',
 'data/chpn-p21.mid',
 'data/chpn-p15.mid',
 'data/chpn-p24.mid',
 'data/chpn_op35_1.mid',
 'data/chpn-p17.mid',
 'data/chpn-p6.mid',
 'data/chpn_op33_4.mid',
 'data/chpn-p11.mid',
 'data/chpn_op10_e12.mid',
 'data/chpn-p22.mid',
 'data/chpn_op10_e01.mid',
 'data/chpn-p23.mid',
 'data/chpn-p2.mid',
 'data/chpn-p18.mid',
 'data/chpn_op33_2.mid',
 'data/chpn-p13.mid',
 'data/chpn_op23.mid',
 'data/chpn_op35_2.mid',
 'data/chpn-p9.mid',
 'data/chpn_op53.mid',
 'data/chpn_op35_3.mid',
 'data/chp_op18.mid',
 'data/chpn-p19.mid',
 'data/chpn-p16.mid',
 'data/chpn-p1.mid',
 'data/chpn_op25_e12.mid',
 'data/chp_op31.mid'

In [ ]:
def get_notes_from_the_files(files):
    """ Get all the notes and chords from the MIDI files in the specified directory """
    notes = []
    

    return notes

In [ ]:
notes_file_path = f'{data_folder}/notes'
notes = []

if os.path.exists(notes_file_path):
   # Loading notes from the file
   with open('data/notes', 'rb') as filepath:
       notes = pickle.load(filepath)
else:
   # creating a list of notes
   notes = get_notes_from_the_files(files)
   # saving notes to the file:
   with open('data/notes', 'wb') as filepath:
       pickle.dump(notes, filepath)

In [ ]:
SEQUENCE_LENGTH = 50  # could be everything from 10 to 100

In [ ]:
# Create input sequences and corresponding output
unique_notes = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(unique_notes))
int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

input_sequences = []
output_sequences = []

for i in range(len(notes) - SEQUENCE_LENGTH):
    sequence_in = notes[i:i + SEQUENCE_LENGTH]
    sequence_out = notes[SEQUENCE_LENGTH]
    input_sequences.append([note_to_int[char] for char in sequence_in])
    output_sequences.append(note_to_int[sequence_out])

## Build and train LSTM model

In [ ]:
EPOCHS = 100  # Adjust as needed
BATCH_SIZE = 128  # Adjust as needed

In [ ]:
# Reshape input sequences
X = np.reshape(input_sequences, (len(input_sequences), SEQUENCE_LENGTH, 1))
X = X / float(len(unique_notes))

# One-hot encode output sequences
y = to_categorical(output_sequences)

model = Sequential()
model.add(LSTM(
        512,
        input_shape=(X.shape[1], X.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

check_point = ModelCheckpoint('model/', save_best_only=True, monitor='loss')
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# Train the model
